### **MonthlyMean snowCover in Fieschertal/Switzerland**

In [ ]:
#Import earth engine
import ee

#Trigger the authentication flow
ee.Authenticate()

#Initialize the library
ee.Initialize(project='modis-410817')

In [ ]:
# Import geemap
import geemap as emap

In [ ]:
# Define the region of interest (ROI)
roi = ee.Geometry.Rectangle([7.948, 46.454, 8.216, 46.563])

# Load MODIS Terra Snow Cover Daily Global 500m
image_collection = ee.ImageCollection("MODIS/061/MOD10A1").select('NDSI_Snow_Cover')

# Filter the image collection for the specified date ranges and region of interest
filtered_collection = (image_collection
                      .filter(ee.Filter.calendarRange(2000, 2020, 'year'))
                      .filter(ee.Filter.Or(
                          ee.Filter.calendarRange(1, 3, 'month'),
                          ee.Filter.calendarRange(10, 12, 'month')
                      ))
                      .filterBounds(roi))

# Define the list of months and years
#months = [1, 2, 3, 10, 11, 12]   ## To get data from 2001 to 2020
months = [ 10, 11, 12]   ## To get data in 2000
#months = [ 1, 2, 3]   ## To get data in 2020

#years = list(range(2001, 2020))   ## To get data from 2001 to 2020
years = list(range(2000, 2001))   ## To get data in 2000
#years = list(range(2020, 2021))   ## To get data in 2020

# Create a list of images by mapping over years and months
image_list = ee.List([
    (filtered_collection
     .filter(ee.Filter.calendarRange(y, y, 'year'))
     .filter(ee.Filter.calendarRange(m, m, 'month'))
     .filterMetadata('system:time_start', 'not_equals', None)  # Filter out images without time_start property
     .mean()
     .set('month', m)
     .set('year', y)
     .set('system:time_start', filtered_collection.first().get('system:time_start'))  # Add time_start property to mean image
    )
    for y in years for m in months
])

# Create an ImageCollection from the list of images
by_month_year = ee.ImageCollection.fromImages(image_list)

In [ ]:
# Get the image IDs
image_ids = by_month_year.aggregate_array('system:index').getInfo()
print('Total images', len(image_ids))

Total images 120


In [ ]:
#Visualization parameters
snow_cover_vis = {
    'min': 0.0,
    'max': 100.0,
    'palette': ['black', '0dffff', '0524ff', 'ffffff']
}

In [ ]:
# Get a list of images
image_list = by_month_year.toList(by_month_year.size())

#Clip for displaying in map
img = ee.Image(image_list.get(0)).clip(roi)

In [ ]:
# Calculate the area of the state
state_area = img.geometry().area()

# Convert the area to square kilometers and round the result
state_area_sq_km = ee.Number(state_area).divide(1e6)

# Print the result
print(state_area_sq_km.getInfo())

248.585614164438


In [ ]:
#Create a center the map object
Map = emap.Map()
Map.center_object(roi, zoom=12)
# Add the layers to the map object.
Map.addLayer(img,snow_cover_vis,'Snow cover')

# Draw the rectangle on the map
Map.addLayer(roi, {'color': 'red'}, 'roi')

#Add a layer control panel to the map
Map.addLayerControl()

#Display the map
Map

Map(center=[46.50856002422174, 8.082000000000294], controls=(WidgetControl(options=['position', 'transparent_b…

In [ ]:
# Get a list of images
image_list = by_month_year.toList(by_month_year.size())

# Loop through the images
for i in range(image_list.length().getInfo()):
    img = ee.Image(image_list.get(i))

    # Cast all bands to Int16
    img = img.toUint16()

    # Extract month and year information from image properties
    month = img.get('month')
    year = img.get('year')

    # Define the export parameters
    export_params = {
        'image': img.select('NDSI_Snow_Cover'),
        'description': 'Snow_cover_Fieschertal_{}_{}_{}'.format(year.getInfo(), month.getInfo(), i + 1),
        'fileNamePrefix': '{}_{}'.format(year.getInfo(), month.getInfo()),  # Include year and month in the file name prefix
        'folder': 'Snow_Cover_Fieschertal',
        'scale': 500,
        'region': roi.getInfo()['coordinates'],
        'crs': 'EPSG:4326',
        'maxPixels': 1e10
    }

    # Start the export task
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()

    print('Started task: ', i + 1)

Started task:  1
Started task:  2
Started task:  3
Started task:  4
Started task:  5
Started task:  6
Started task:  7
Started task:  8
Started task:  9
Started task:  10
Started task:  11
Started task:  12
Started task:  13
Started task:  14
Started task:  15
Started task:  16
Started task:  17
Started task:  18
Started task:  19
Started task:  20
Started task:  21
Started task:  22
Started task:  23
Started task:  24
Started task:  25
Started task:  26
Started task:  27
Started task:  28
Started task:  29
Started task:  30
Started task:  31
Started task:  32
Started task:  33
Started task:  34
Started task:  35
Started task:  36
Started task:  37
Started task:  38
Started task:  39
Started task:  40
Started task:  41
Started task:  42
Started task:  43
Started task:  44
Started task:  45
Started task:  46
Started task:  47
Started task:  48
Started task:  49
Started task:  50
Started task:  51
Started task:  52
Started task:  53
Started task:  54
Started task:  55
Started task:  56
S